In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_scar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 22:01:54,897] A new study created in memory with name: no-name-37ad9fee-880f-4bf7-9590-7b67e2bc2290


Running experiment with seed=654:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.13932336327408168, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.003087514626074824
Epoch 0 / 100, Loss: 27.7274
Epoch 10 / 100, Loss: 21.4100
Epoch 20 / 100, Loss: 16.7578
Epoch 30 / 100, Loss: 12.9332
Epoch 40 / 100, Loss: 10.0351
Epoch 50 / 100, Loss: 7.9643
Epoch 60 / 100, Loss: 6.3783
Epoch 70 / 100, Loss: 5.2735
Epoch 80 / 100, Loss: 4.3328
Epoch 90 / 100, Loss: 3.6551
 - Metrics: Accuracy=0.9369, F1=0.8456, Recall=0.8203, Precision=0.8725
Running experiment with seed=114:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.13932336327408168, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=75
 - model_

[I 2025-03-26 22:08:54,753] Trial 0 finished with value: 0.8453960694225009 and parameters: {'K': 7, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.13932336327408168, 'model_type': 'GCNConv', 'batch_size': 256, 'lr': 0.003087514626074824, 'clusters': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 75}. Best is trial 0 with value: 0.8453960694225009.


 - Metrics: Accuracy=0.9324, F1=0.8356, Recall=0.8160, Precision=0.8563
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2603220154.csv.
Average F1 over valid seeds: 0.8454 ± 0.0063
Running experiment with seed=654:
 - K=2, layers=2, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.13750087548779888, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=2, clusters=300, lr=0.0015207494162327235
Epoch 0 / 100, Loss: 2.2770
Epoch 10 / 100, Loss: 2.0525
Epoch 20 / 100, Loss: 2.0233
Epoch 30 / 100, Loss: 1.8897
Epoch 40 / 100, Loss: 1.8259
Epoch 50 / 100, Loss: 1.7592
Epoch 60 / 100, Loss: 1.7085
Epoch 70 / 100, Loss: 1.6272
Epoch 80 / 100, Loss: 1.5657
Epoch 90 / 100, Loss: 1.5083


[I 2025-03-26 22:09:32,505] Trial 1 finished with value: 0.8216216216216217 and parameters: {'K': 2, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.13750087548779888, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0015207494162327235, 'clusters': 300, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8453960694225009.


...

 - Metrics: Accuracy=0.9420, F1=0.8571, Recall=0.8260, Precision=0.8908
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090009.csv.
Average F1 over valid seeds: 0.8614 ± 0.0073
Running experiment with seed=654:
 - K=3, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11266072511121758, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.007405345109839189
Epoch 0 / 100, Loss: 3.0388
Epoch 10 / 100, Loss: 2.4135
Epoch 20 / 100, Loss: 1.9736
Epoch 30 / 100, Loss: 1.6323
Epoch 40 / 100, Loss: 1.2881
Epoch 50 / 100, Loss: 1.0654
Epoch 60 / 100, Loss: 0.8707
Epoch 70 / 100, Loss: 0.7608
Epoch 80 / 100, Loss: 0.6406
Epoch 90 / 100, Loss: 0.5536
 - Metrics: Accuracy=0.9519, F1=0.8815, Recall=0.8488, Precision=0.9168
Running experiment with seed=114:
 - K=3, layers=1, hidden=64, out=256
 

[I 2025-03-27 09:05:47,916] Trial 198 finished with value: 0.8702456042121606 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.11266072511121758, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.007405345109839189, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 156 with value: 0.8722422037353572.


 - Metrics: Accuracy=0.9468, F1=0.8694, Recall=0.8402, Precision=0.9006
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090241.csv.
Average F1 over valid seeds: 0.8702 ± 0.0059
Running experiment with seed=654:
 - K=3, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11442266685450465, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.0074238320254282454
Epoch 0 / 100, Loss: 3.0382
Epoch 10 / 100, Loss: 2.4131
Epoch 20 / 100, Loss: 1.9727
Epoch 30 / 100, Loss: 1.6299
Epoch 40 / 100, Loss: 1.2857
Epoch 50 / 100, Loss: 1.0629
Epoch 60 / 100, Loss: 0.8686
Epoch 70 / 100, Loss: 0.7585
Epoch 80 / 100, Loss: 0.6390
Epoch 90 / 100, Loss: 0.5519
 - Metrics: Accuracy=0.9498, F1=0.8773, Recall=0.8516, Precision=0.9045
Running experiment with seed=114:
 - K=3, layers=1, hidden=64, out=256


[I 2025-03-27 09:08:56,552] Trial 199 finished with value: 0.8693388634016216 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.11442266685450465, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0074238320254282454, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 156 with value: 0.8722422037353572.


 - Metrics: Accuracy=0.9444, F1=0.8631, Recall=0.8317, Precision=0.8969
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_2703090547.csv.
Average F1 over valid seeds: 0.8693 ± 0.0055
Best trial:
  Average F1: 0.8722422037353572
  Best parameters:
    K: 3
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.14573085963762972
    model_type: GATConv
    batch_size: 1024
    lr: 0.006079262879388443
    clusters: 100
    sampling: neighbor
    sampling_k: 75


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-25 10:10:43,640] A new study created in memory with name: no-name-f25fde7d-ee4a-4fa2-be4f-f9e486b8970e


Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=50, lr=0.0023234250431828636
Epoch 0 / 100, Loss: 2.8385
Epoch 10 / 100, Loss: 2.4108
Epoch 20 / 100, Loss: 2.2628
Epoch 30 / 100, Loss: 2.1825
Epoch 40 / 100, Loss: 2.0409
Epoch 50 / 100, Loss: 1.9945
Epoch 60 / 100, Loss: 1.8232
Epoch 70 / 100, Loss: 1.6971
Epoch 80 / 100, Loss: 1.6468
Epoch 90 / 100, Loss: 1.4848
 - Metrics: Accuracy=0.9405, F1=0.8507, Recall=0.8046, Precision=0.9024
Running experiment with seed=114:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pa

[I 2025-03-25 10:13:49,885] Trial 0 finished with value: 0.857595508989174 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.11307195638376069, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0023234250431828636, 'anomaly_detector': 'nearest_neighbors', 'clusters': 50, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.857595508989174.


 - Metrics: Accuracy=0.9390, F1=0.8493, Recall=0.8160, Precision=0.8854
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503101043.csv.
Average F1 over valid seeds: 0.8576 ± 0.0077
Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.17225386531043968, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=400, lr=0.008752505664905935
Epoch 0 / 150, Loss: 42.4167
Epoch 10 / 150, Loss: 18.6557
Epoch 20 / 150, Loss: 10.0401
Epoch 30 / 150, Loss: 6.0609
Epoch 40 / 150, Loss: 4.2251
Epoch 50 / 150, Loss: 3.0423
Epoch 60 / 150, Loss: 2.2833
Epoch 70 / 150, Loss: 1.8259
Epoch 80 / 150, Loss: 1.4801
Epoch 90 / 150, Loss: 1.2157
Epoch 100 / 150, Loss: 1.0309
Epoch 110 / 150, Loss: 0.8732
Epoch 120 / 150, Loss: 0.7590
Epoch 130 / 150, Loss: 0.6525
Epoch 140 / 150, Loss

[I 2025-03-25 10:15:40,677] Trial 1 finished with value: 0.6813996316758748 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.17225386531043968, 'model_type': 'GATConv', 'batch_size': 256, 'lr': 0.008752505664905935, 'anomaly_detector': 'unweighted', 'clusters': 400, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.857595508989174.


...

 - Metrics: Accuracy=0.9059, F1=0.7183, Recall=0.5692, Precision=0.9732
F1 = 0.72 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140907.csv.
Average F1 over valid seeds: 0.7183 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15841141409782936, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004733199465670689
Epoch 0 / 100, Loss: 2.1699
Epoch 10 / 100, Loss: 1.6532
Epoch 20 / 100, Loss: 1.4729
Epoch 30 / 100, Loss: 1.3237
Epoch 40 / 100, Loss: 1.1338
Epoch 50 / 100, Loss: 1.0336
Epoch 60 / 100, Loss: 0.8867
Epoch 70 / 100, Loss: 0.7703
Epoch 80 / 100, Loss: 0.7087
Epoch 90 / 100, Loss: 0.6093
 - Metrics: Accuracy=0.9408, F1=0.8600, Recall=0.8631, Precision=0.8569
Running experiment with seed=114:
 - K=2

[I 2025-03-25 14:12:40,226] Trial 98 finished with value: 0.8617602498928539 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.15841141409782936, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004733199465670689, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9387, F1=0.8539, Recall=0.8502, Precision=0.8576
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140941.csv.
Average F1 over valid seeds: 0.8618 ± 0.0066
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16102039029803536, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004605584968207975
Epoch 0 / 100, Loss: 4.1609
Epoch 10 / 100, Loss: 3.4473
Epoch 20 / 100, Loss: 3.0725
Epoch 30 / 100, Loss: 2.6641
Epoch 40 / 100, Loss: 2.3129
Epoch 50 / 100, Loss: 2.1049
Epoch 60 / 100, Loss: 1.7786
Epoch 70 / 100, Loss: 1.6623
Epoch 80 / 100, Loss: 1.4632
Epoch 90 / 100, Loss: 1.3323


[I 2025-03-25 14:13:15,258] Trial 99 finished with value: 0.8429629629629629 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16102039029803536, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004605584968207975, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9363, F1=0.8430, Recall=0.8117, Precision=0.8767
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503141240.csv.
Average F1 over valid seeds: 0.8430 ± 0.0000
Best trial:
  Average F1: 0.8650138572197479
  Best parameters:
    K: 2
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.1611386183555754
    model_type: GCNConv
    batch_size: 1024
    lr: 0.0063212510924867535
    anomaly_detector: nearest_neighbors
    clusters: 200
    num_epochs: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_scar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-27 10:52:51,733] A new study created in memory with name: no-name-0e26df09-7fa5-46d1-be19-f32a53fd7c0d


Running experiment with seed=654:
 - K=10, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.19622450292840055, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.003767773886669508


### Hyperparameter Optimization Cora
#### SAR 

In [5]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 12, 16),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-21 16:27:22,810] A new study created in memory with name: no-name-1b6493b1-fec6-42d0-9fb9-895b7ae8f2c2


Running experiment with seed=654:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.24996292612162108, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=10, clusters=200, lr=0.008750241270040128
Epoch 0 / 100, Loss: 94.6581
Epoch 10 / 100, Loss: 14.4191
Epoch 20 / 100, Loss: 4.8927
Epoch 30 / 100, Loss: 2.5103
Epoch 40 / 100, Loss: 1.5205
Epoch 50 / 100, Loss: 1.0131
Epoch 60 / 100, Loss: 0.7141
Epoch 70 / 100, Loss: 0.5221
Epoch 80 / 100, Loss: 0.3939
Epoch 90 / 100, Loss: 0.3018
 - Metrics: Accuracy=0.8871, F1=0.8529, Recall=0.8194, Precision=0.8892
Running experiment with seed=114:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.24996292612162108, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_

[I 2025-03-21 18:08:01,436] Trial 0 finished with value: 0.8491476722544864 and parameters: {'K': 12, 'ratio': 0.24996292612162108, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.008750241270040128, 'anomaly_detector': 'nearest_neighbors', 'clusters': 200, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8491476722544864.


 - Metrics: Accuracy=0.8818, F1=0.8465, Recall=0.8163, Precision=0.8791
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_2103162722.csv.
Average F1 over valid seeds: 0.8491 ± 0.0022
Running experiment with seed=654:
 - K=16, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.3028969707946579, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=400, lr=0.008614726546514963


Computing METIS partitioning...
Done!


...

 - Metrics: Accuracy=0.8724, F1=0.8399, Recall=0.8380, Precision=0.8418
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_2403030315.csv.
Average F1 over valid seeds: 0.8504 ± 0.0059
Running experiment with seed=654:
 - K=15, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.33466954295031853, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=10, clusters=400, lr=0.009449005803853418
Epoch 0 / 100, Loss: 36.5753
Epoch 10 / 100, Loss: 13.9073
Epoch 20 / 100, Loss: 4.5616
Epoch 30 / 100, Loss: 2.3028
Epoch 40 / 100, Loss: 1.4206
Epoch 50 / 100, Loss: 0.9757
Epoch 60 / 100, Loss: 0.7131
Epoch 70 / 100, Loss: 0.5453
Epoch 80 / 100, Loss: 0.4286
Epoch 90 / 100, Loss: 0.3449
 - Metrics: Accuracy=0.8838, F1=0.8560, Recall=0.8650, Precision=0.8472
Running experiment with seed=114:
 - K=1

[I 2025-03-24 05:35:00,309] Trial 98 finished with value: 0.8542428905078753 and parameters: {'K': 15, 'ratio': 0.33466954295031853, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.009449005803853418, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 92 with value: 0.8550235782168171.


 - Metrics: Accuracy=0.8791, F1=0.8513, Recall=0.8664, Precision=0.8368
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_2403043706.csv.
Average F1 over valid seeds: 0.8542 ± 0.0034
Running experiment with seed=654:
 - K=15, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3346057738503619, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=10, clusters=200, lr=0.0008260446599945397
Epoch 0 / 150, Loss: 37.4467
Epoch 10 / 150, Loss: 35.8211
Epoch 20 / 150, Loss: 34.1562
Epoch 30 / 150, Loss: 32.3217
Epoch 40 / 150, Loss: 30.4127
Epoch 50 / 150, Loss: 28.6562
Epoch 60 / 150, Loss: 26.2881
Epoch 70 / 150, Loss: 24.1003
Epoch 80 / 150, Loss: 21.7394
Epoch 90 / 150, Loss: 19.5714
Epoch 100 / 150, Loss: 17.5102
Epoch 110 / 150, Loss: 15.7757
Epoch 120 / 150, Loss: 14.0564
Epoch 130 / 150, Loss: 12.5691
Epoch 140 /

[I 2025-03-24 05:51:18,301] Trial 99 finished with value: 0.8105207367165761 and parameters: {'K': 15, 'ratio': 0.3346057738503619, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0008260446599945397, 'anomaly_detector': 'nearest_neighbors', 'clusters': 200, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 92 with value: 0.8550235782168171.


 - Metrics: Accuracy=0.8549, F1=0.8105, Recall=0.7768, Precision=0.8473
F1 = 0.81 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scar_sampling_nnif_batch_cluster_2403053500.csv.
Average F1 over valid seeds: 0.8105 ± 0.0000
Best trial:
  Average F1: 0.8550235782168171
  Best parameters:
    K: 15
    ratio: 0.327995195770344
    model_type: GATConv
    batch_size: 1024
    lr: 0.009721945078527111
    anomaly_detector: nearest_neighbors
    clusters: 400
    num_epochs: 100
    sampling: neighbor
    sampling_k: 75


### Hyperparameter Optimization Pubmed
#### SAR 

In [11]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 8),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 128,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-14 02:04:35,449] A new study created in memory with name: no-name-e5dcf471-fcac-4aab-8f6d-b3b093666d23


Running experiment with seed=654:
 - K=8, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.11008449308317064, aggregation=mean, treatment=removal
 - model_type=GCNConv, rate_pairs=3, clusters=100, lr=0.001451234283197598


Computing METIS partitioning...
Done!


Epoch 0, Loss: 33.5343
Epoch 10, Loss: 29.7977
Epoch 20, Loss: 27.8999
Epoch 30, Loss: 23.6593
Epoch 40, Loss: 19.9143
 - Metrics: Accuracy=0.9574, F1=0.9014, Recall=0.8499, Precision=0.9595
Running experiment with seed=114:
 - K=8, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=5, reliable_mini_batch=True
 - ratio=0.11008449308317064, aggregation=mean, treatment=removal
 - model_type=GCNConv, rate_pairs=3, clusters=100, lr=0.001451234283197598


Computing METIS partitioning...
Done!


...

Computing METIS partitioning...
Done!


Epoch 0, Loss: 20.9089
Epoch 10, Loss: 19.5268
Epoch 20, Loss: 16.6540
Epoch 30, Loss: 15.6314
Epoch 40, Loss: 14.6815


[I 2025-03-14 14:57:53,054] Trial 199 finished with value: 0.9197740892805021 and parameters: {'K': 35, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16810415247537985, 'aggregation': 'mean', 'treatment': 'removal', 'model_type': 'GCNConv', 'rate_pairs': 7, 'batch_size': 15, 'lr': 0.0025304429458340204, 'clusters': 100}. Best is trial 163 with value: 0.9244167623455898.


 - Metrics: Accuracy=0.9634, F1=0.9204, Recall=0.9242, Precision=0.9167
Done. Results written to wiki-cs_experimentations\wikics_scar_sampling_nnif_batch_cluster_1403145358.csv.
Average F1 over valid seeds: 0.9198 ± 0.0010
Best trial:
  Average F1: 0.9244167623455898
  Best parameters:
    K: 34
    layers: 2
    hidden_channels: 256
    out_channels: 256
    ratio: 0.17331567524348823
    aggregation: mean
    treatment: removal
    model_type: GCNConv
    rate_pairs: 8
    batch_size: 15
    lr: 0.0018990508573961478
    clusters: 100


: 

### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [13]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 3, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [10,20]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster","nearest_neighbors"]),         
        "sampling_k":trial.suggest_categorical("sampling_k",[25,50,75]),
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 20:02:21,721] A new study created in memory with name: no-name-9c84a830-1d32-4a81-aa10-5679b684cd4d


Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0.1983021209568396, batch_size=10, methodology=ours
 - ratio=0.01676656982996787, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=neighbor, sampling_k=50
 - model_type=GATConv, rate_pairs=9, clusters=1000, lr=0.007076470456182623


[I 2025-03-24 20:02:40,646] Trial 0 finished with value: 0.0 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'dropout': 0.1983021209568396, 'ratio': 0.01676656982996787, 'model_type': 'GATConv', 'rate_pairs': 9, 'batch_size': 10, 'lr': 0.007076470456182623, 'anomaly_detector': 'unweighted', 'clusters': 1000, 'num_epochs': 50, 'sampling': 'neighbor', 'sampling_k': 50}. Best is trial 0 with value: 0.0.


Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2403200221.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=128, out=256
 - norm=None, dropout=0.01605654337952993, batch_size=20, methodology=ours
 - ratio=0.024315606511573024, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=50
 - model_type=GATConv, rate_pairs=7, clusters=1000, lr=0.0089094751164754


[I 2025-03-25 01:09:57,013] Trial 1 finished with value: 0.0 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.01605654337952993, 'ratio': 0.024315606511573024, 'model_type': 'GATConv', 'rate_pairs': 7, 'batch_size': 20, 'lr': 0.0089094751164754, 'anomaly_detector': 'unweighted', 'clusters': 1000, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 0 with value: 0.0.


...

Running experiment with seed=654:
 - K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.06199985776750046, batch_size=10, methodology=ours
 - ratio=0.022437251981747064, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GATConv, rate_pairs=9, clusters=500, lr=0.007917920503781244
Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2503010957.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000


[I 2025-03-25 01:10:21,032] Trial 3 finished with value: 0.0 and parameters: {'K': 5, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.03228429726090343, 'ratio': 0.008491919490729881, 'model_type': 'GCNConv', 'rate_pairs': 6, 'batch_size': 20, 'lr': 0.008387901223664842, 'anomaly_detector': 'unweighted', 'clusters': 500, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 0 with value: 0.0.


Running experiment with seed=654:
 - K=5, layers=2, hidden=256, out=128
 - norm=None, dropout=0.03228429726090343, batch_size=20, methodology=ours
 - ratio=0.008491919490729881, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=6, clusters=500, lr=0.008387901223664842
Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_2503011009.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=17, layers=1, hidden=128, out=128
 - norm=None, dropout=0.2915360402823104, batch_size=20, methodology=ours
 - ratio=0.020473770474233224, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=50
 - model_type=GCNConv, rate_pairs=3, clusters=1000, lr=0.009469031504558686


Computing METIS partitioning...


: 